In [172]:
# !pip install librosa

In [173]:
# !pip install kagglehub
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("uwrfkaggler/ravdess-emotional-speech-audio")

# print("Path to dataset files:", path)
# # C:\Users\<Username>\.cache\kagglehub\datasets\uwrfkaggler\ravdess-emotional-speech-audio\versions\1

In [174]:
from pathlib import Path

DATASET_ROOT = Path(
    r"C:\Users\DipeshBindlish\.cache\kagglehub\datasets\uwrfkaggler\ravdess-emotional-speech-audio\versions\1"
    # "/mnt/c/Users/DipeshBindlish/.cache/kagglehub/datasets/uwrfkaggler/ravdess-emotional-speech-audio/versions/1"
)

print(DATASET_ROOT)
print("Exists:", DATASET_ROOT.exists())

C:\Users\DipeshBindlish\.cache\kagglehub\datasets\uwrfkaggler\ravdess-emotional-speech-audio\versions\1
Exists: True


In [175]:
for p in DATASET_ROOT.iterdir():
    print(p)

ACTORS_DIR = DATASET_ROOT / "Audio_Speech_Actors_01-24"
for p in list(ACTORS_DIR.iterdir())[:5]:
    print(p)


C:\Users\DipeshBindlish\.cache\kagglehub\datasets\uwrfkaggler\ravdess-emotional-speech-audio\versions\1\Actor_01
C:\Users\DipeshBindlish\.cache\kagglehub\datasets\uwrfkaggler\ravdess-emotional-speech-audio\versions\1\Actor_02
C:\Users\DipeshBindlish\.cache\kagglehub\datasets\uwrfkaggler\ravdess-emotional-speech-audio\versions\1\Actor_03
C:\Users\DipeshBindlish\.cache\kagglehub\datasets\uwrfkaggler\ravdess-emotional-speech-audio\versions\1\Actor_04
C:\Users\DipeshBindlish\.cache\kagglehub\datasets\uwrfkaggler\ravdess-emotional-speech-audio\versions\1\Actor_05
C:\Users\DipeshBindlish\.cache\kagglehub\datasets\uwrfkaggler\ravdess-emotional-speech-audio\versions\1\Actor_06
C:\Users\DipeshBindlish\.cache\kagglehub\datasets\uwrfkaggler\ravdess-emotional-speech-audio\versions\1\Actor_07
C:\Users\DipeshBindlish\.cache\kagglehub\datasets\uwrfkaggler\ravdess-emotional-speech-audio\versions\1\Actor_08
C:\Users\DipeshBindlish\.cache\kagglehub\datasets\uwrfkaggler\ravdess-emotional-speech-audio\ver

In [176]:
audio_files = list(ACTORS_DIR.rglob("*.wav"))

print("Total audio files:", len(audio_files))
print("Sample file:", audio_files[0])

Total audio files: 1440
Sample file: C:\Users\DipeshBindlish\.cache\kagglehub\datasets\uwrfkaggler\ravdess-emotional-speech-audio\versions\1\Audio_Speech_Actors_01-24\Actor_01\03-01-01-01-01-01-01.wav


In [177]:
emotion_map = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

In [178]:
def parse_emotion(path: Path):
    parts = path.stem.split("-")
    emotion_code = parts[2]
    return emotion_map[emotion_code]

test_file = audio_files[0]
print(test_file.name)
print("Parsed emotion:", parse_emotion(test_file))


03-01-01-01-01-01-01.wav
Parsed emotion: neutral


In [179]:
samples = []

for f in audio_files:
    samples.append({
        "path": str(f),
        "emotion": parse_emotion(f)
    })

print("Total samples:", len(samples))
print("First 3 samples:")
samples[:3]

Total samples: 1440
First 3 samples:


[{'path': 'C:\\Users\\DipeshBindlish\\.cache\\kagglehub\\datasets\\uwrfkaggler\\ravdess-emotional-speech-audio\\versions\\1\\Audio_Speech_Actors_01-24\\Actor_01\\03-01-01-01-01-01-01.wav',
  'emotion': 'neutral'},
 {'path': 'C:\\Users\\DipeshBindlish\\.cache\\kagglehub\\datasets\\uwrfkaggler\\ravdess-emotional-speech-audio\\versions\\1\\Audio_Speech_Actors_01-24\\Actor_01\\03-01-01-01-01-02-01.wav',
  'emotion': 'neutral'},
 {'path': 'C:\\Users\\DipeshBindlish\\.cache\\kagglehub\\datasets\\uwrfkaggler\\ravdess-emotional-speech-audio\\versions\\1\\Audio_Speech_Actors_01-24\\Actor_01\\03-01-01-01-02-01-01.wav',
  'emotion': 'neutral'}]

In [180]:
import soundfile as sf

sample = samples[0]
waveform, sr = sf.read(sample["path"])

print("Emotion:", sample["emotion"])
print("Waveform shape:", waveform.shape)
print("Sampling rate:", sr)
print("First 10 values:", waveform[:10])
for i, j in enumerate(waveform):
    if j != 0.0:
        print(f"{i}: {j}")
        print(waveform[i-5:i+5])
        break

Emotion: neutral
Waveform shape: (158558,)
Sampling rate: 48000
First 10 values: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
38: 3.0517578125e-05
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 3.05175781e-05 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]


In [181]:
import librosa

if sr != 16000:
    waveform_16k = librosa.resample(
        waveform,
        orig_sr=sr,
        target_sr=16000
    )
    sr = 16000
else:
    waveform_16k = waveform

print("Resampled shape:", waveform_16k.shape)
print("New sampling rate:", sr)
print("First 10 samples:", waveform_16k[:10])

Resampled shape: (52853,)
New sampling rate: 16000
First 10 samples: [ 1.40786398e-07 -1.92843316e-07  2.51211077e-07 -3.16654933e-07
  3.90405802e-07 -4.74502087e-07  5.72436363e-07 -6.90470756e-07
  8.40631174e-07 -1.04856667e-06]


In [182]:
from IPython.display import Audio, display

display(Audio(waveform_16k, rate=sr))

In [183]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained(
    "facebook/wav2vec2-base"
)

c:\Users\DipeshBindlish\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [184]:
inputs = processor(
    waveform_16k,
    sampling_rate=sr,
    return_tensors="pt",
    padding=True
)

for k, v in inputs.items():
    print(k, v.shape, v.dtype)

input_values torch.Size([1, 52853]) torch.float32


In [185]:
labels = sorted(set(s["emotion"] for s in samples))
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}

print("Labels:", labels)
print("Label → ID:", label2id)

Labels: ['angry', 'calm', 'disgust', 'fearful', 'happy', 'neutral', 'sad', 'surprised']
Label → ID: {'angry': 0, 'calm': 1, 'disgust': 2, 'fearful': 3, 'happy': 4, 'neutral': 5, 'sad': 6, 'surprised': 7}


In [186]:
from datasets import Dataset

dataset = Dataset.from_list(samples)
print(dataset)

Dataset({
    features: ['path', 'emotion'],
    num_rows: 1440
})


In [187]:
def encode_labels(example):
    example["labels"] = label2id[example["emotion"]]
    return example

dataset = dataset.map(encode_labels)
print(dataset[0])

Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

{'path': 'C:\\Users\\DipeshBindlish\\.cache\\kagglehub\\datasets\\uwrfkaggler\\ravdess-emotional-speech-audio\\versions\\1\\Audio_Speech_Actors_01-24\\Actor_01\\03-01-01-01-01-01-01.wav', 'emotion': 'neutral', 'labels': 5}


In [188]:
dataset = dataset.train_test_split(test_size=0.2, seed=42)
print(dataset)

# Reduce dataset size for quicker testing
dataset["train"] = dataset["train"].shuffle(seed=42).select(range(10))
dataset["test"]  = dataset["test"].shuffle(seed=42).select(range(4))

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['path', 'emotion', 'labels'],
        num_rows: 1152
    })
    test: Dataset({
        features: ['path', 'emotion', 'labels'],
        num_rows: 288
    })
})
DatasetDict({
    train: Dataset({
        features: ['path', 'emotion', 'labels'],
        num_rows: 10
    })
    test: Dataset({
        features: ['path', 'emotion', 'labels'],
        num_rows: 4
    })
})


In [189]:
import soundfile as sf
import librosa

def preprocess_audio(batch):
    waveforms = []

    for path in batch["path"]:
        waveform, sr = sf.read(path)

        # convert stereo → mono
        if waveform.ndim > 1:
            waveform = waveform.mean(axis=1)

        # resample
        if sr != 16000:
            waveform = librosa.resample(
                waveform,
                orig_sr=sr,
                target_sr=16000
            )

        waveforms.append(waveform)

    inputs = processor(
        waveforms,
        sampling_rate=16000,
        padding=True,
        return_tensors="pt"
    )

    inputs["labels"] = batch["labels"]
    return inputs

In [190]:
dataset = dataset.map(
    preprocess_audio,
    batched=True,
    batch_size=8,
    remove_columns=["path", "emotion"]
)

print(dataset)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_values'],
        num_rows: 10
    })
    test: Dataset({
        features: ['labels', 'input_values'],
        num_rows: 4
    })
})


In [191]:
print(dataset["train"][0].keys())

for k, v in dataset["train"][0].items():
    if hasattr(v, "shape"):
        print(k, v.shape)
    else:
        print(k, v)

dict_keys(['labels', 'input_values'])
labels 1
input_values [0.017480913549661636, 0.0361265167593956, 0.037916768342256546, 0.03405376523733139, 0.03200366720557213, 0.03204909339547157, 0.036040663719177246, 0.03484078124165535, 0.030715499073266983, 0.024011418223381042, 0.032627999782562256, 0.02150786481797695, 0.02907782606780529, 0.03989753499627113, 0.03721826523542404, 0.043204210698604584, 0.03021164797246456, 0.031379956752061844, 0.028856879100203514, 0.026510292664170265, 0.02807021699845791, 0.029413005337119102, 0.033471863716840744, 0.03907471150159836, 0.034907493740320206, 0.03263182193040848, 0.03173036128282547, 0.03408615663647652, 0.03810032829642296, 0.0360332652926445, 0.03634382039308548, 0.03832101449370384, 0.03114442713558674, 0.03669579327106476, 0.034273017197847366, 0.03242725133895874, 0.04655290022492409, 0.03222210332751274, 0.03624862805008888, 0.037089236080646515, 0.044161729514598846, 0.04685094207525253, 0.0335996150970459, 0.036051712930202484, 0

In [192]:
dataset.set_format("torch")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_values'],
        num_rows: 10
    })
    test: Dataset({
        features: ['labels', 'input_values'],
        num_rows: 4
    })
})


In [193]:
from transformers import Wav2Vec2ForSequenceClassification

num_labels = len(label2id)

model = Wav2Vec2ForSequenceClassification.from_pretrained(
    "facebook/wav2vec2-base",
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label
)

print(model)

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2ForSequenceClassification(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)


In [194]:
model.freeze_feature_extractor()

c:\Users\DipeshBindlish\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:2306: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [195]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="audio_out",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    # dataloader_num_workers=2,  # speed up data loading
    # dataloader_pin_memory=True,  # speed up data transfer to GPU
    num_train_epochs=10,
    logging_steps=10,
    fp16=False,  # set True only if GPU supports it
    report_to=[],
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

In [196]:
import numpy as np
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

In [197]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(
    tokenizer=processor,
    padding=True,
    return_tensors="pt"
)

In [198]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

C:\Users\DipeshBindlish\AppData\Local\Temp\ipykernel_15380\237722441.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [199]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.071825,0.250000
2,No log,2.078879,0.250000
3,No log,2.065863,0.250000
4,No log,2.070832,0.250000
5,1.413500,2.072272,0.250000


c:\Users\DipeshBindlish\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\DipeshBindlish\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\DipeshBindlish\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\DipeshBindlish\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device

TrainOutput(global_step=10, training_loss=1.4135424613952636, metrics={'train_runtime': 119.8681, 'train_samples_per_second': 0.834, 'train_steps_per_second': 0.083, 'total_flos': 1984195887055200.0, 'train_loss': 1.4135424613952636, 'epoch': 5.0})

In [200]:
metrics = trainer.evaluate()
print(metrics)

c:\Users\DipeshBindlish\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 2.071824789047241, 'eval_accuracy': 0.25, 'eval_runtime': 1.8169, 'eval_samples_per_second': 2.202, 'eval_steps_per_second': 0.55, 'epoch': 5.0}
